In [2]:
from collections import defaultdict
from glob import glob
import os
import pandas as pd
import statistics as stats
import numpy as np
import math
import random
import matplotlib.pyplot as plt
from matplotlib import cm
import itertools

In [3]:
project_dir = '/Users/tonyb/Documents/Networking_Pilot/'
raw_dir = project_dir + 'all_data/'
explore_dir = project_dir + 'explore_n33/'
if not os.path.exists(explore_dir): os.makedirs(explore_dir)

In [4]:
abbrev = {'directed_forgetting_condition':'DF',
#           'directed_condition': 'DF',
         'flanker_condition':'FLANKER',
         'go_nogo_condition':'GNG',
         'n_back_condition':'NBACK',
          'delay_condition':'DELAY',
         'predictable_condition':'PREDICT',
#           'predictive_condition':'PREDICT',
         'shape_matching_condition':'SHAPE',
         'stop_signal_condition':'SS',
         'cue-task_condition':'TASK',
         'cue_condition':'CUE',
         'cued_condition':'CUE'}

In [5]:
explore_id=['A3QAHF4UUBM7ZO',
'AIQT0DPRTXYYD',
'A3V1ZYXBJYCIIE',
'A2WYCY1FMQOD5F',
'A2S4YDJ9UGAXFQ',
'A3G55RJTW3BSGM',
'A1OSRAPSRT934Z',
'AVJUIF9QHQRY8',
'A1VCAMP3XM62R4',
'A3IW9415ZOO0EX',
'A55CXM7QR7R0N',
'A2XUADP5L61HQ5',
'AQL960O0LTRI8',
'A1DS5O8MSI3ZH0',
'A2YTO4EY3MNYAJ',
'AY7WPVKHVNBLG',
'A2OFN0A5CPLH57',
'A2JI5RNPPXE8QE',
'A1TS2SKXPX7ZED',
'A1YC558J4E5KZ',
'A2DWPP1KKAY0HG',
'AD7CUW86FWEKT',
'A2SIKW18T2DYYW',
'AQJWO4YPR3LUQ',
'A1L1SQ488YCCFJ',
'A3ISDIYTS02E8C',
'A3QJJR5Y3XE92N',
'A1R0689JPSQ3OF',
'A37EV8RZ82WT8E',
'A3GEHH49HNJM57',
'ASTR3EPUOKEXV',
'AVMIXXCHPD291',
'A2581F7TDPAMBQ',]

In [ ]:
### check for inconsistent naming among tasks and missing of necessary columns.
predictive = []
directed = []
currenttrial = []
task = []

for subj_file in glob(raw_dir + 'A*/*'):
    filey = os.path.basename(subj_file) #filey is now cued_task_switching_with_directed_forgetting_A2YBZFSUD5OP7W.csv or any other subid_taskid.csv
    if ('predictive_condition' in df.columns) or ('predictive_dimension' in df.columns):
        predictive.append(exp_id)
    if 'directed_condition' in df.columns:
        directed.append(exp_id)
    if 'current_trial' not in df.columns and 'trial_index' in df.columns:
        currenttrial.append(exp_id)
    if 'task_condition' in df.columns:
        task.append(exp_id)
set(task)

### Generate concat data from raw data

In [ ]:
task_dfs = defaultdict(pd.DataFrame)
for subj_file in glob(raw_dir + 'A*/*'):
    filey = os.path.basename(subj_file) #e.g., cued_task_switching_single_task_network_A1DS5O8MSI3ZH0.csv
    worker_id = filey.split('_')[-1].split('.')[0] #filey.split('_')[-1] returns 'A2581F7TDPAMBQ.csv'; filey.split('_')[-1].split('.')[0] returns 'A2581F7TDPAMBQ'
    
    if worker_id in explore_id:
        df = pd.read_csv(subj_file)
        if 'exp_id' in df.columns:
            exp_id = df.exp_id.unique()[0] #for the network_attack battery, all rows in the exp_id column has the same value
        else:
            exp_id = '_'.join(filey.split('_')[0:-1]).rstrip('.csv')
        if 'worker_id' not in df.columns:
            df.loc[:,'worker_id'] = worker_id
        # really avoid any malfunctioning correct_trial
        if 'correct_trial' in df.columns and 'key_press'in df.columns and 'correct_response' in df.columns:
            df.rename(columns={'correct_trial':'original_correcttrial'},inplace=True)
            df['correct_trial'] = np.where(df['key_press']==df['correct_response'],1,0) #np.where(condition,if True,if False)
        # post process data, drop rows, etc.....
        for colname in df.columns:
            if 'predictive_condition' in colname:
                df.rename(columns={colname:'predictable_condition'},inplace=True)
            if 'predictive_dimension' in colname:
                df.rename(columns={colname:'predictable_dimension'},inplace=True)
            if 'directed_condition' in colname:
                df.rename(columns={colname:'directed_forgetting_condition'},inplace=True)
            #change to cue-task_condition to avoid mistaking cued-task-switching's "task_condition"
            #with other "task_condition"s in other tasks.
            if 'task_condition' in colname:
                df.rename(columns={colname:'cue-task_condition'},inplace=True)
            if ('SS' not in colname) and ('delay' in colname):
                df.rename(columns={colname:'delay_condition'},inplace=True)
        drop_columns = ['Unnamed: 0','final_accuracy','final_avg_rt', 'final_credit_var','final_missed_percent',
                        'final_responses_ok','internal_node_id','responses','stimulus',
                        'text','trial_index','trial_type','view_history']
        df.drop(drop_columns, axis=1, inplace=True, errors='ignore')
        keep_dict = {'trial_id': ['test_trial']}
        #somehow query() would not run if a column was read first (even though that column existed in the df) 
        #hence, all the following if statements
        if 'trial_id' in df.columns:
            for row, vals in keep_dict.items():
                df = df.query('%s in %s' % (row, vals))
        if 'predictable' in exp_id or 'predictive' in exp_id:
            if 'current_trial' in df.columns:
                df = df.query('current_trial > 1') #remove first trial for all predictable-task switching tasks
            elif 'current_trial' not in df.columns:
                if 'cue-task_condition' in df.columns:
                    df = df[-((pd.isna(df['cue-task_condition'])) | (df['cue-task_condition']=="na"))]
                elif 'predictable_condition' in df.columns:
                    df = df[-((pd.isna(df['predictable_condition'])) | (df['predictable_condition']=="na"))]
                elif 'predictive_condition' in df.columns:
                    df = df[-((pd.isna(df['predictive_condition'])) | (df['predictive_condition']=="na"))]
        if 'n_back' in exp_id and 'current_trial' in df.columns:
            df = df.query('current_trial > 3') #remove first 3 trials for all n_back tasks        
        if 'cue' in exp_id and 'cue-task_condition' in df.columns:
            df = df[-((pd.isna(df['cue-task_condition'])) | (df['cue-task_condition']=="na"))] #remove first trial, 'na' or null, for all cued-task switching tasks
        task_dfs[exp_id] = pd.concat([task_dfs[exp_id], df], axis=0)

if not os.path.exists(explore_dir + 'concat/'): os.makedirs(explore_dir + 'concat/')
for task,df in task_dfs.items():
    if 'demographics' not in task: #because demographic data files have different structures than behavioral data files
        df.to_csv(explore_dir + 'concat/' + task + '.csv', index=False)

In [ ]:
### check which tasks had malfunctioning correct_trial (that's automated by javascript)
for concat_file in glob(explore_dir + 'concat/*'):
    filey = os.path.basename(concat_file)
    concat_df = pd.read_csv(concat_file)
    if 'correct_trial' in df.columns and 'original_correcttrial'in df.columns: #'original_correcttrial' generated in the preceding cell
        new_mean = stats.mean(concat_df['correct_trial'])
        original_mean = stats.mean(concat_df['original_correcttrial'])
        if new_mean != original_mean:
            print(filey)

### Generate descriptive stats (qa files) from concat data

In [15]:
for concat_file in glob(explore_dir + 'concat/*'):
    filey = os.path.basename(concat_file) #filey is now cued_task_switching_with_directed_forgetting_A2YBZFSUD5OP7W.csv or any other taskid_subid.csv
    concat_df = pd.read_csv(concat_file)
    #get task name
    exp_id = concat_df.exp_id.unique()[0] #for the network_attack battery, all rows in the exp_id column has the same value
    # collapsing across H and F in flanker_condition
    if ('flanker' in exp_id) and (len(concat_df['flanker_condition'].unique())>2):
        concat_df = concat_df.replace({'flanker_condition':[r'(^.*inc.*$)',r'(^.*_con.*$)']},\
                          {'flanker_condition':['incongruent','congruent']},regex=True)
    # collapsing 7 levels in shape_matching_condition: noise-response-incompatible (i.e. distractor != target, or D in second letter) vs. no-noise (i.e. no distractor, or N in second letter); also, disregarding SSS & DSD
    if ('shape' in exp_id) and (len(concat_df['shape_matching_condition'].unique())>2):
        concat_df = concat_df.replace({'shape_matching_condition':['SDD','DDS','DDD','D!=T']},\
                          {'shape_matching_condition':['DISTRACTOR','DISTRACTOR','DISTRACTOR','DISTRACTOR']})
        concat_df = concat_df.replace({'shape_matching_condition':['SNN','DNN','NoD']},\
                          {'shape_matching_condition':['CONTROL','CONTROL','CONTROL']})
    
    ###prep for generation of colnames of qa_df
    qa_colnames = []
#     conditions = [i for i in concat_df.columns if ('condition' in i)]
    #use the following line instead of the preceding one if only care about number of delays (1,2,3) instead of n_back_condition (match, mismatch)
    conditions = [i for i in concat_df.columns if ('condition' in i) and ('n_back_condition' not in i)]
    conditions.sort()
    # prep for calculation of SSRT (only for stop-signal tasks): replace and rank GoRT:
    if ('stop' in exp_id) and ('stop_signal_condition' in concat_df.columns):
        concat_df['GoRTWithReplacement'] = np.where(concat_df['stop_signal_condition']=='go',np.where(concat_df['rt']== -1,2000,concat_df['rt']), np.nan)
        concat_df['GoRTWithReplacementRank'] = concat_df.groupby(['worker_id']+conditions, sort=False)['GoRTWithReplacement'].rank(ascending=True, method='first')
        concat_df.drop([i for i in concat_df.columns if 'Unnamed' in i], axis=1, inplace=True, errors='ignore')
        concat_df.to_csv(concat_file,index=False)
    
    
    ### Tasks with 1 condition variable
    if len(conditions) == 1:
        ordered_condition0 = concat_df[conditions[0]].unique()
        ordered_condition0.sort()
        for condition0 in ordered_condition0:
            condition_name = str(abbrev[conditions[0]] + ':' + str(condition0)) #TASK:stay
            if 'SS:go' in condition_name:
                qa_colnames.append(str('countGoTrials_' + condition_name))
            elif 'SS:stop' in condition_name:
                for i in ['ssd_', 'ssrt_', 'm(stopfail_rt)_', 'sd(stopfail_rt)_', 'commission_']:
                    qa_colname = str(i + condition_name)
                    qa_colnames.append(qa_colname)
            elif 'GNG:nogo' in condition_name:
                for i in ['m(nogofail_rt)_', 'sd(nogofail_rt)_', 'commission_']:
                    qa_colname = str(i + condition_name)
                    qa_colnames.append(qa_colname)
            else:
                for i in ['m(rt)_','sd(rt)_','acc_']:
                    qa_colname = str(i + condition_name) #m(rt)_TASK:stay
                    qa_colnames.append(qa_colname)
        qa_df = pd.DataFrame(index=concat_df.worker_id.unique(),columns=qa_colnames)
        for condition0 in ordered_condition0:
            condition_name = str(abbrev[conditions[0]] + ':' + str(condition0))
            for worker_id in qa_df.index:
                ### df by worker_id to calculate descriptive stats
                df_subj = concat_df[(concat_df['worker_id']==worker_id) & (concat_df[conditions[0]]==condition0)]
                ### number of go trials
                if ('stop' in exp_id) and ('SS:go' in condition_name):
                    countGoTrials = str('countGoTrials_'+condition_name)
                    qa_df.at[worker_id,countGoTrials] = len(df_subj) #len(df_subj): all Go trials 
                ### mean response times
                mrt = str('m(rt)_'+condition_name)
                if len(df_subj['rt'][(df_subj['correct_trial']==1)])==0:
                    qa_df.at[worker_id,mrt] = np.nan
                else:
                    mean_rt = stats.mean(df_subj['rt'][(df_subj['correct_trial']==1)])
                    qa_df.at[worker_id,mrt] = round(mean_rt,2)
                ### sd response times
                sdrt = str('sd(rt)_'+condition_name)
                if len(df_subj['rt'][(df_subj['correct_trial']==1)])<2:
                    qa_df.at[worker_id,sdrt] = np.nan
                else:
                    sd_rt = stats.stdev(df_subj['rt'][(df_subj['correct_trial']==1)])
                    qa_df.at[worker_id,sdrt] = round(sd_rt,2)
                ### accuracy rate, i.e., count of correct responses
                #keep the else statement only if NOT adding 0.5 error for those with accuracy = 1
                if len(df_subj[df_subj['correct_trial']==1]) == len(df_subj):
                    accuracy = (len(df_subj[df_subj['correct_trial']==1]) - 0.5)/len(df_subj)
                else:
                    accuracy = len(df_subj[df_subj['correct_trial']==1])/len(df_subj)
                acc = str('acc_'+condition_name)
                qa_df.at[worker_id,acc] = accuracy
                if ('_no' in exp_id) and (condition0 == 'nogo'):
                    if 'go_nogo_condition' in df_subj.columns:
                        df_subj_nogo = df_subj[(df_subj['go_nogo_condition']=='nogo')]
                        # commission rate
                        nogofail = len(df_subj_nogo[df_subj_nogo['correct_trial']==0])/len(df_subj_nogo)
                        nogo_fail = str('commission_'+condition_name)
                        qa_df.at[worker_id,nogo_fail] = round(nogofail,2)
                        #m(rt_nogofail)
                        mrt_nogofail = str('m(nogofail_rt)_'+condition_name)    
                        if len(df_subj_nogo['rt'][(df_subj_nogo['correct_trial']==0)])==0:
                            qa_df.at[worker_id,mrt_nogofail] = np.nan
                        else:
                            mrtfail = stats.mean(df_subj_nogo['rt'][(df_subj_nogo['correct_trial']==0)])
                            qa_df.at[worker_id,mrt_nogofail] = round(mrtfail,2)
                        #sd(rt_nogofail)
                        sdrt_nogofail = str('sd(nogofail_rt)_'+condition_name)
                        if len(df_subj_nogo['rt'][(df_subj_nogo['correct_trial']==0)])<2:
                            qa_df.at[worker_id,sdrt_nogofail] = np.nan
                        else:
                            sdrtfail = stats.stdev(df_subj_nogo['rt'][(df_subj_nogo['correct_trial']==0)])
                            qa_df.at[worker_id,sdrt_nogofail] = round(sdrtfail,2)
                    else:
                        print('go_nogo_condition NOT in '+exp_id+'.columns')
                if ('stop' in exp_id) and (condition0 == 'stop'):
                    if 'stop_signal_condition' in df_subj.columns and 'SS_delay' in df_subj.columns:
                        df_subj_stop = df_subj[(df_subj['stop_signal_condition']=='stop')]
                        # ssd
                        ssd = stats.mean(df_subj_stop['SS_delay'])
                        SS_delay = str('ssd_'+condition_name)
                        qa_df.at[worker_id,SS_delay] = round(ssd,2)
                        # commission rate
                        stopfail = len(df_subj_stop[df_subj_stop['correct_trial']==0])/len(df_subj_stop)
                        stop_fail = str('commission_'+condition_name)
                        qa_df.at[worker_id,stop_fail] = round(stopfail,2)
                        #ssrt
                        rank = int(qa_df.at[worker_id,[i for i in qa_df.columns if ('countGoTrials_' in i) & ('SS:go' in i)][0]]*stopfail)
                        nthRT = list(concat_df['GoRTWithReplacement'][(concat_df['worker_id']==worker_id) & (concat_df['GoRTWithReplacementRank']==rank)])[0]
                        ssrt = nthRT - ssd
                        SSRT = str('ssrt_'+condition_name)
                        qa_df.at[worker_id,SSRT] = round(ssrt,2)
                        #m(rt_stopfail)
                        mrt_stopfail = str('m(stopfail_rt)_'+condition_name)    
                        if len(df_subj_stop['rt'][(df_subj_stop['correct_trial']==0)])==0:
                            qa_df.at[worker_id,mrt_stopfail] = np.nan
                        else:
                            mrtfail = stats.mean(df_subj_stop['rt'][(df_subj_stop['correct_trial']==0)])
                            qa_df.at[worker_id,mrt_stopfail] = round(mrtfail,2)
                        #sd(rt_stopfail)
                        sdrt_stopfail = str('sd(stopfail_rt)_'+condition_name)
                        if len(df_subj_stop['rt'][(df_subj_stop['correct_trial']==0)])<2:
                            qa_df.at[worker_id,sdrt_stopfail] = np.nan
                        else:
                            sdrtfail = stats.stdev(df_subj_stop['rt'][(df_subj_stop['correct_trial']==0)])
                            qa_df.at[worker_id,sdrt_stopfail] = round(sdrtfail,2)
                    else:
                        print('stop_signal_condition and SS_delay NOT in '+exp_id+'.columns')
    
    
    ### Tasks with 2 condition variables
    elif len(conditions) == 2:
        ordered_condition0 = concat_df[conditions[0]].unique()
        ordered_condition0.sort()
        ordered_condition1 = concat_df[conditions[1]].unique()
        ordered_condition1.sort()
        for condition0 in ordered_condition0:
            for condition1 in ordered_condition1:
                condition_name = str(abbrev[conditions[0]] + ':' + str(condition0) + '_&_' + abbrev[conditions[1]] + ':' + str(condition1)) #TASK:stay_&_GNG:go
                if 'SS:go' in condition_name:
                    qa_colnames.append(str('countGoTrials_' + condition_name))
                elif 'SS:stop' in condition_name:
                    for i in ['ssd_', 'ssrt_', 'm(stopfail_rt)_', 'sd(stopfail_rt)_', 'commission_']:
                        qa_colname = str(i + condition_name)
                        qa_colnames.append(qa_colname)
                elif 'GNG:nogo' in condition_name:
                    for i in ['m(nogofail_rt)_', 'sd(nogofail_rt)_', 'commission_']:
                        qa_colname = str(i + condition_name)
                        qa_colnames.append(qa_colname)
                else:
                    for i in ['m(rt)_','sd(rt)_','acc_']:
                        qa_colname = str(i + condition_name) #m(rt)_TASK:stay
                        qa_colnames.append(qa_colname)
        qa_df = pd.DataFrame(index=concat_df.worker_id.unique(),columns=qa_colnames)
        for condition0 in ordered_condition0:
            for condition1 in ordered_condition1:
                if 'stop' not in conditions[0]:
                    no_stop_condition_name = str(abbrev[conditions[0]] + ':' + str(condition0))
                else:
                    print('stop_signal_condition NOT last in conditions')
                condition_name = str(abbrev[conditions[0]] + ':' + str(condition0) + '_&_' + abbrev[conditions[1]] + ':' + str(condition1)) #TASK:stay_&_GNG:go
                for worker_id in qa_df.index:
                    df_subj = concat_df[(concat_df['worker_id']==worker_id) & (concat_df[conditions[0]]==condition0) & (concat_df[conditions[1]]==condition1)]
                    # number of go trials
                    if ('stop' in exp_id) and ('SS:go' in condition_name):
                        countGoTrials = str('countGoTrials_'+condition_name)
                        qa_df.at[worker_id,countGoTrials] = len(df_subj)  #len(df_subj): all Go trials in one of the other condition's levels
                    # mean response times
                    mrt = str('m(rt)_'+condition_name)
                    if len(df_subj['rt'][(df_subj['correct_trial']==1)])==0:
#                         print(worker_id + ' had 0 correct trial in ' + condition_name + ' in ' + exp_id)
                        qa_df.at[worker_id,mrt] = np.nan
                    else:
                        mean_rt = round(stats.mean(df_subj['rt'][(df_subj['correct_trial']==1)]),2)
                        qa_df.at[worker_id,mrt] = mean_rt
                    # sd response times
                    sdrt = str('sd(rt)_'+condition_name)
                    if len(df_subj['rt'][(df_subj['correct_trial']==1)])<2:
                        qa_df.at[worker_id,sdrt] = np.nan
                    else:
                        sd_rt = round(stats.stdev(df_subj['rt'][(df_subj['correct_trial']==1)]),2)
                        qa_df.at[worker_id,sdrt] = sd_rt
                    # accuracy rate, i.e., count of correct responses
                    if len(df_subj) !=0:
                        #keep the else statement only if NOT adding 0.5 error for those with accuracy = 1
                        if len(df_subj[df_subj['correct_trial']==1]) == len(df_subj):
                            accuracy = (len(df_subj[df_subj['correct_trial']==1]) - 0.5)/len(df_subj)
                        else:
                            accuracy = len(df_subj[df_subj['correct_trial']==1])/len(df_subj)
                        acc = str('acc_'+condition_name)
                        qa_df.at[worker_id,acc] = accuracy
                    if '_no' in exp_id and condition0 == 'nogo' or condition1 == 'nogo' and 'go_nogo_condition' in df_subj.columns:
                        df_subj_nogo = df_subj[(df_subj['go_nogo_condition']=='nogo')]
                        # commission rate
                        if len(df_subj_nogo) !=0:
                            nogofail = round(len(df_subj_nogo[df_subj_nogo['correct_trial']==0])/len(df_subj_nogo),2)
                            nogo_fail = str('commission_'+condition_name)
                            qa_df.at[worker_id,nogo_fail] = nogofail
                        else:
                            qa_df.at[worker_id,nogo_fail] = np.nan
                        #m(rt_nogofail)
                        mrt_nogofail = str('m(nogofail_rt)_'+condition_name)
                        if len(df_subj_nogo['rt'][(df_subj_nogo['correct_trial']==0)])==0:
                            qa_df.at[worker_id,mrt_nogofail] = np.nan
                        else:
                            mrtfail = round(stats.mean(df_subj_nogo['rt'][(df_subj_nogo['correct_trial']==0)]),2)
                            qa_df.at[worker_id,mrt_nogofail] = mrtfail
                        #sd(rt_nogofail)
                        sdrt_nogofail = str('sd(nogofail_rt)_'+condition_name)
                        if len(df_subj_nogo['rt'][(df_subj_nogo['correct_trial']==0)])<2:
                            qa_df.at[worker_id,sdrt_nogofail] = np.nan
                        else:
                            sdrtfail = round(stats.stdev(df_subj_nogo['rt'][(df_subj_nogo['correct_trial']==0)]),2)
                            qa_df.at[worker_id,sdrt_nogofail] = sdrtfail
                    if 'stop' in exp_id and condition0 == 'stop' or condition1 == 'stop' and 'stop_signal_condition' in df_subj.columns and 'SS_delay' in df_subj.columns:
                        df_subj_stop = df_subj[(df_subj['stop_signal_condition']=='stop')]
                        # ssd
                        if len(df_subj_stop['SS_delay']) >1:
                            ssd = round(stats.mean(df_subj_stop['SS_delay']),2)
                            SS_delay = str('ssd_'+condition_name)
                            qa_df.at[worker_id,SS_delay] = ssd
                        else:
                            qa_df.at[worker_id,SS_delay] = np.nan
                        # commission rate
                        if len(df_subj_stop) !=0:
                            stopfail = round(len(df_subj_stop[df_subj_stop['correct_trial']==0])/len(df_subj_stop),2)
                            stop_fail = str('commission_'+condition_name)
                            qa_df.at[worker_id,stop_fail] = stopfail
                        else:
                            qa_df.at[worker_id,stop_fail] = np.nan
                        #ssrt
                        SSRT = str('ssrt_'+condition_name)
                        rank = int(qa_df.at[worker_id,[i for i in qa_df.columns if ('countGoTrials_' in i) & (no_stop_condition_name in i) & ('SS:go' in i)][0]]*stopfail)
                        if rank >0:
                            nthRT = list(concat_df['GoRTWithReplacement'][(concat_df['worker_id']==worker_id) & (concat_df[conditions[0]]==condition0) & (concat_df['GoRTWithReplacementRank']==rank)])[0]
                            ssrt = nthRT - ssd
                            qa_df.at[worker_id,SSRT] = round(ssrt,2)
                        else:
                            qa_df.at[worker_id,SSRT] = 'rank <0'
                        #m(rt_stopfail)
                        mrt_stopfail = str('m(stopfail_rt)_'+condition_name)
                        if len(df_subj_stop['rt'][(df_subj_stop['correct_trial']==0)])==0:
                            qa_df.at[worker_id,mrt_stopfail] = np.nan
                        else:
                            mrtfail = round(stats.mean(df_subj_stop['rt'][(df_subj_stop['correct_trial']==0)]),2)
                            qa_df.at[worker_id,mrt_stopfail] = mrtfail
                        #sd(rt_stopfail)
                        sdrt_stopfail = str('sd(stopfail_rt)_'+condition_name)
                        if len(df_subj_stop['rt'][(df_subj_stop['correct_trial']==0)])<2:
                            qa_df.at[worker_id,sdrt_stopfail] = np.nan
                        else:
                            sdrtfail = round(stats.stdev(df_subj_stop['rt'][(df_subj_stop['correct_trial']==0)]),2)
                            qa_df.at[worker_id,sdrt_stopfail] = sdrtfail


    ### DUAL-TASKs with 3 _condition variables
    elif len(conditions) == 3:
        ordered_condition0 = concat_df[conditions[0]].unique()
        ordered_condition0.sort()
        ordered_condition1 = concat_df[conditions[1]].unique()
        ordered_condition1.sort()
        ordered_condition2 = concat_df[conditions[2]].unique()
        ordered_condition2.sort()
        for condition0 in ordered_condition0:
            for condition1 in ordered_condition1:
                for condition2 in ordered_condition2:
                    condition_name = str(abbrev[conditions[0]] + ':' + str(condition0) + '_&_' + abbrev[conditions[1]] + ':' + str(condition1) + '_&_' + abbrev[conditions[2]] + ':' + str(condition2)) #TASK:stay_&_GNG:go
                    if 'SS:go' in condition_name:
                        qa_colnames.append(str('countGoTrials_' + condition_name))
                    elif 'SS:stop' in condition_name:
                        for i in ['ssd_', 'ssrt_', 'm(stopfail_rt)_', 'sd(stopfail_rt)_', 'commission_']:
                            qa_colname = str(i + condition_name)
                            qa_colnames.append(qa_colname)
                    elif 'GNG:nogo' in condition_name:
                        for i in ['m(nogofail_rt)_', 'sd(nogofail_rt)_', 'commission_']:
                            qa_colname = str(i + condition_name)
                            qa_colnames.append(qa_colname)
                    else:
                        for i in ['m(rt)_','sd(rt)_','acc_']:
                            qa_colname = str(i + condition_name)
                            qa_colnames.append(qa_colname)
        qa_df = pd.DataFrame(index=concat_df.worker_id.unique(),columns=qa_colnames)
        for condition0 in ordered_condition0:
            for condition1 in ordered_condition1:
                for condition2 in ordered_condition2:
                    if ('stop' not in conditions[0]) and ('stop' not in conditions[1]):
                        no_stop_condition_name = str(abbrev[conditions[0]] + ':' + str(condition0) + '_&_' + abbrev[conditions[1]] + ':' + str(condition1))
                    else:
                        print('stop_signal_condition is NOT last in conditions')
                    condition_name = str(abbrev[conditions[0]] + ':' + str(condition0) + '_&_' + abbrev[conditions[1]] + ':' + str(condition1) + '_&_' + abbrev[conditions[2]] + ':' + str(condition2)) #TASK:stay_&_GNG:go
                    
                    for worker_id in qa_df.index:
                        df_subj = concat_df[(concat_df['worker_id']==worker_id) & (concat_df[conditions[0]]==condition0) & (concat_df[conditions[1]]==condition1) & (concat_df[conditions[2]]==condition2)]
                        # number of go trials
                        if ('stop' in exp_id) and ('SS:go' in condition_name):
                            countGoTrials = str('countGoTrials_'+condition_name)
                            qa_df.at[worker_id,countGoTrials] = len(df_subj)  #len(df_subj): all Go trials in one of the other condition's levels
                        # mean response times
                        mrt = str('m(rt)_'+condition_name)
                        if len(df_subj['rt'][(df_subj['correct_trial']==1)])==0:
                            qa_df.at[worker_id,mrt] = np.nan
                        else:
                            mean_rt = round(stats.mean(df_subj['rt'][(df_subj['correct_trial']==1)]),2)
                            qa_df.at[worker_id,mrt] = mean_rt
                        # sd response times
                        sdrt = str('sd(rt)_'+condition_name)
                        if len(df_subj['rt'][(df_subj['correct_trial']==1)])<2:
                            qa_df.at[worker_id,sdrt] = np.nan
                        else:
                            sd_rt = round(stats.stdev(df_subj['rt'][(df_subj['correct_trial']==1)]),2)
                            qa_df.at[worker_id,sdrt] = sd_rt
                        # accuracy rate, i.e., count of correct responses
                        if len(df_subj) !=0:
                            #keep the else statement only if NOT adding 0.5 error for those with accuracy = 1
                            if len(df_subj[df_subj['correct_trial']==1]) == len(df_subj):
                                accuracy = (len(df_subj[df_subj['correct_trial']==1]) - 0.5)/len(df_subj)
                            else:
                                accuracy = len(df_subj[df_subj['correct_trial']==1])/len(df_subj)
                            acc = str('acc_'+condition_name)
                            qa_df.at[worker_id,acc] = accuracy
                        if '_no' in exp_id and condition0 == 'nogo' or condition1 == 'nogo' or condition2 == 'nogo' and 'go_nogo_condition' in df_subj.columns:
                            df_subj_nogo = df_subj[(df_subj['go_nogo_condition']=='nogo')]
                            # commission rate
                            if len(df_subj_nogo) !=0:
                                nogofail = round(len(df_subj_nogo[df_subj_nogo['correct_trial']==0])/len(df_subj_nogo),2)
                                nogo_fail = str('commission_'+condition_name)
                                qa_df.at[worker_id,nogo_fail] = nogofail
                            #m(rt_nogofail)
                            mrt_nogofail = str('m(nogofail_rt)_'+condition_name)
                            if len(df_subj_nogo['rt'][(df_subj_nogo['correct_trial']==0)])==0:
                                qa_df.at[worker_id,mrt_nogofail] = np.nan
                            else:
                                mrtfail = round(stats.mean(df_subj_nogo['rt'][(df_subj_nogo['correct_trial']==0)]),2)
                                qa_df.at[worker_id,mrt_nogofail] = mrtfail
                            #sd(rt_nogofail)
                            sdrt_nogofail = str('sd(nogofail_rt)_'+condition_name)
                            if len(df_subj_nogo['rt'][(df_subj_nogo['correct_trial']==0)])<2:
                                qa_df.at[worker_id,sdrt_nogofail] = np.nan
                            else:
                                sdrtfail = round(stats.stdev(df_subj_nogo['rt'][(df_subj_nogo['correct_trial']==0)]),2)
                                qa_df.at[worker_id,sdrt_nogofail] = sdrtfail
                        if 'stop' in exp_id and condition0 == 'stop' or condition1 == 'stop' or condition2 == 'stop' and 'stop_signal_condition' in df_subj.columns and 'SS_delay' in df_subj.columns:
                            df_subj_stop = df_subj[(df_subj['stop_signal_condition']=='stop')]
                            # ssd
                            if len(df_subj_stop['SS_delay']) >1:
                                ssd = round(stats.mean(df_subj_stop['SS_delay']),2)
                                SS_delay = str('ssd_'+condition_name)
                                qa_df.at[worker_id,SS_delay] = ssd
                            # commission rate
                            if len(df_subj_stop) !=0:
                                stopfail = round(len(df_subj_stop[df_subj_stop['correct_trial']==0])/len(df_subj_stop),2)
                                stop_fail = str('commission_'+condition_name)
                                qa_df.at[worker_id,stop_fail] = stopfail
                            #ssrt
                            SSRT = str('ssrt_'+condition_name)
                            rank = int(qa_df.at[worker_id,[i for i in qa_df.columns if ('countGoTrials_' in i) & (no_stop_condition_name in i) & ('SS:go' in i)][0]]*stopfail)
                            if rank >0:
                                nthRT = list(concat_df['GoRTWithReplacement'][(concat_df['worker_id']==worker_id) & (concat_df[conditions[0]]==condition0) & (concat_df[conditions[1]]==condition1) & (concat_df['GoRTWithReplacementRank']==rank)])[0]
                                ssrt = nthRT - ssd
                                qa_df.at[worker_id,SSRT] = round(ssrt,2)
                            else:
                                qa_df.at[worker_id,SSRT] = 'rank <0'
                            #m(rt_stopfail)
                            mrt_stopfail = str('m(stopfail_rt)_'+condition_name)
                            if len(df_subj_stop['rt'][(df_subj_stop['correct_trial']==0)])==0:
                                qa_df.at[worker_id,mrt_stopfail] = np.nan
                            else:
                                mrtfail = round(stats.mean(df_subj_stop['rt'][(df_subj_stop['correct_trial']==0)]),2)
                                qa_df.at[worker_id,mrt_stopfail] = mrtfail
                            #sd(rt_stopfail)
                            sdrt_stopfail = str('sd(stopfail_rt)_'+condition_name)
                            if len(df_subj_stop['rt'][(df_subj_stop['correct_trial']==0)])<2:
                                qa_df.at[worker_id,sdrt_stopfail] = np.nan
                            else:
                                sdrtfail = round(stats.stdev(df_subj_stop['rt'][(df_subj_stop['correct_trial']==0)]),2)
                                qa_df.at[worker_id,sdrt_stopfail] = sdrtfail
   
    
    ### DUAL-TASKs with 4 _condition variables
    ### n_back_with_cued_task_switching is the only case ('TASK','CUE','NBACK','DELAY')
    ### if conditions = [...] is defined to exclude n_back_condition, then there's none.
    elif len(conditions) == 4:
        ordered_condition0 = concat_df[conditions[0]].unique()
        ordered_condition0.sort()
        ordered_condition1 = concat_df[conditions[1]].unique()
        ordered_condition1.sort()
        ordered_condition2 = concat_df[conditions[2]].unique()
        ordered_condition2.sort()
        ordered_condition3 = concat_df[conditions[3]].unique()
        ordered_condition3.sort()
        for condition0 in ordered_condition0:
            for condition1 in ordered_condition1:
                for condition2 in ordered_condition2:
                    for condition3 in ordered_condition3:
                        condition_name = str(abbrev[conditions[0]] + ':' + str(condition0) + '_&_' + abbrev[conditions[1]] + ':' + str(condition1) + '_&_' + abbrev[conditions[2]] + ':' + str(condition2) + '_&_' + abbrev[conditions[3]] + ':' + str(condition3)) #TASK:stay_&_GNG:go
                        if 'SS:go' in condition_name:
                            qa_colnames.append(str('countGoTrials_' + condition_name))
                        elif 'SS:stop' in condition_name:
                            for i in ['ssd_', 'ssrt_', 'm(stopfail_rt)_', 'sd(stopfail_rt)_', 'commission_']:
                                qa_colname = str(i + condition_name)
                                qa_colnames.append(qa_colname)
                        elif 'GNG:nogo' in condition_name:
                            for i in ['m(nogofail_rt)_', 'sd(nogofail_rt)_', 'commission_']:
                                qa_colname = str(i + condition_name)
                                qa_colnames.append(qa_colname)
                        else:
                            for i in ['m(rt)_','sd(rt)_','acc_']:
                                qa_colname = str(i + condition_name)
                                qa_colnames.append(qa_colname)
        qa_df = pd.DataFrame(index=concat_df.worker_id.unique(),columns=qa_colnames)
        for condition0 in ordered_condition0:
            for condition1 in ordered_condition1:
                for condition2 in ordered_condition2:
                    for condition3 in ordered_condition3:
                        if ('stop' not in conditions[0]) and ('stop' not in conditions[1]) and ('stop' not in conditions[2]):
                            no_stop_condition_name = str(abbrev[conditions[0]] + ':' + str(condition0) + '_&_' + abbrev[conditions[1]] + ':' + str(condition1) + '_&_' + abbrev[conditions[2]] + ':' + str(condition2))
                        else:
                            print('stop_signal_condition is NOT last in conditions')
                        condition_name = str(abbrev[conditions[0]] + ':' + str(condition0) + '_&_' + abbrev[conditions[1]] + ':' + str(condition1) + '_&_' + abbrev[conditions[2]] + ':' + str(condition2) + '_&_' + abbrev[conditions[3]] + ':' + str(condition3)) #TASK:stay_&_GNG:go
                        for worker_id in qa_df.index:
                            df_subj = concat_df[(concat_df['worker_id']==worker_id) & (concat_df[conditions[0]]==condition0) & (concat_df[conditions[1]]==condition1) & (concat_df[conditions[2]]==condition2) & (concat_df[conditions[3]]==condition3)]
                            # number of go trials
                            if ('stop' in exp_id) and ('SS:go' in condition_name):
                                countGoTrials = str('countGoTrials_'+condition_name)
                                qa_df.at[worker_id,countGoTrials] = len(df_subj)  #len(df_subj): all Go trials in one of the other condition's levels
                            # mean response times
                            mrt = str('m(rt)_'+condition_name)
                            if len(df_subj['rt'][(df_subj['correct_trial']==1)])==0:
                                qa_df.at[worker_id,mrt] = np.nan
                            else:
                                mean_rt = round(stats.mean(df_subj['rt'][(df_subj['correct_trial']==1)]),2)
                                qa_df.at[worker_id,mrt] = mean_rt
                            # sd response times
                            sdrt = str('sd(rt)_'+condition_name)
                            if len(df_subj['rt'][(df_subj['correct_trial']==1)])<2:
                                qa_df.at[worker_id,sdrt] = np.nan
                            else:
                                sd_rt = round(stats.stdev(df_subj['rt'][(df_subj['correct_trial']==1)]),2)
                                qa_df.at[worker_id,sdrt] = sd_rt
                            # accuracy rate, i.e., count of correct responses
                            if len(df_subj) !=0:
                                #keep the else statement only if NOT adding 0.5 error for those with accuracy = 1
                                if len(df_subj[df_subj['correct_trial']==1]) == len(df_subj):
                                    accuracy = (len(df_subj[df_subj['correct_trial']==1]) - 0.5)/len(df_subj)
                                else:
                                    accuracy = len(df_subj[df_subj['correct_trial']==1])/len(df_subj)
                                acc = str('acc_'+condition_name)
                                qa_df.at[worker_id,acc] = accuracy
                            if '_no' in exp_id and condition0 == 'nogo' or condition1 == 'nogo' or condition2 == 'nogo' or condition3 == 'nogo' and 'go_nogo_condition' in df_subj.columns:
                                df_subj_nogo = df_subj[(df_subj['go_nogo_condition']=='nogo')]
                                # commission rate
                                if len(df_subj_nogo) !=0:
                                    nogofail = round(len(df_subj_nogo[df_subj_nogo['correct_trial']==0])/len(df_subj_nogo),2)
                                    nogo_fail = str('commission_'+condition_name)
                                    qa_df.at[worker_id,nogo_fail] = nogofail
                                else:
                                    qa_df.at[worker_id,nogo_fail] = np.nan
                                #m(rt_nogofail)
                                mrt_nogofail = str('m(nogofail_rt)_'+condition_name)
                                if len(df_subj_nogo['rt'][(df_subj_nogo['correct_trial']==0)])==0:
                                    qa_df.at[worker_id,mrt_nogofail] = np.nan
                                else:
                                    mrtfail = round(stats.mean(df_subj_nogo['rt'][(df_subj_nogo['correct_trial']==0)]),2)
                                    qa_df.at[worker_id,mrt_nogofail] = mrtfail
                                #sd(rt_nogofail)
                                sdrt_nogofail = str('sd(nogofail_rt)_'+condition_name)
                                if len(df_subj_nogo['rt'][(df_subj_nogo['correct_trial']==0)])<2:
                                    qa_df.at[worker_id,sdrt_nogofail] = np.nan
                                else:
                                    sdrtfail = round(stats.stdev(df_subj_nogo['rt'][(df_subj_nogo['correct_trial']==0)]),2)
                                    qa_df.at[worker_id,sdrt_nogofail] = sdrtfail
                            if 'stop' in exp_id and condition0 == 'stop' or condition1 == 'stop' or condition2 == 'stop' or condition3 == 'stop' and 'stop_signal_condition' in df_subj.columns and 'SS_delay' in df_subj.columns:
                                df_subj_stop = df_subj[(df_subj['stop_signal_condition']=='stop')]
                                # ssd
                                if len(df_subj_stop['SS_delay']) >1:
                                    ssd = round(stats.mean(df_subj_stop['SS_delay']),2)
                                    SS_delay = str('ssd_'+condition_name)
                                    qa_df.at[worker_id,SS_delay] = ssd
                                else:
                                    qa_df.at[worker_id,SS_delay] = np.nan
                                # commission rate
                                if len(df_subj_stop) !=0:
                                    stopfail = round(len(df_subj_stop[df_subj_stop['correct_trial']==0])/len(df_subj_stop),2)
                                    stop_fail = str('commission_'+condition_name)
                                    qa_df.at[worker_id,stop_fail] = stopfail
                                else:
                                    qa_df.at[worker_id,stop_fail] = np.nan
                                #ssrt
                                SSRT = str('ssrt_'+condition_name)
                                rank = int(qa_df.at[worker_id,[i for i in qa_df.columns if ('countGoTrials_' in i) & (no_stop_condition_name in i) & ('SS:go' in i)][0]]*stopfail)
                                if rank >0:
                                    nthRT = list(concat_df['GoRTWithReplacement'][(concat_df['worker_id']==worker_id) & (concat_df[conditions[0]]==condition0) & (concat_df[conditions[1]]==condition1) & (concat_df[conditions[2]]==condition2) & (concat_df['GoRTWithReplacementRank']==rank)])[0]
                                    ssrt = nthRT - ssd
                                    qa_df.at[worker_id,SSRT] = round(ssrt,2)
                                else:
                                    qa_df.at[worker_id,SSRT] = 'rank <0'
                                #m(rt_stopfail)
                                mrt_stopfail = str('m(stopfail_rt)_'+condition_name)
                                if len(df_subj_stop['rt'][(df_subj_stop['correct_trial']==0)])==0:
                                    qa_df.at[worker_id,mrt_stopfail] = np.nan
                                else:
                                    mrtfail = round(stats.mean(df_subj_stop['rt'][(df_subj_stop['correct_trial']==0)]),2)
                                    qa_df.at[worker_id,mrt_stopfail] = mrtfail
                                #sd(rt_stopfail)
                                sdrt_stopfail = str('sd(stopfail_rt)_'+condition_name)
                                if len(df_subj_stop['rt'][(df_subj_stop['correct_trial']==0)])<2:
                                    qa_df.at[worker_id,sdrt_stopfail] = np.nan
                                else:
                                    sdrtfail = round(stats.stdev(df_subj_stop['rt'][(df_subj_stop['correct_trial']==0)]),2)
                                    qa_df.at[worker_id,sdrt_stopfail] = sdrtfail

    
    ###drop some stats automatically generated that are redundant and DVs not of interest before writing out QA data frames
    #DVs:
    qa_df.drop([i for i in qa_df.columns if ('TASK:switch' in i) & ('CUE:stay' in i)], axis=1, inplace=True, errors='ignore')
    qa_df.drop([i for i in qa_df.columns if 'DF:pos' in i], axis=1, inplace=True, errors='ignore')
    qa_df.drop([i for i in qa_df.columns if ('SSS' in i) | ('DSD' in i)], axis=1, inplace=True, errors='ignore')
    #stats:
    qa_df.drop([i for i in qa_df.columns if ('countGoTrials' in i) & ('SS:stop' in i)], axis=1, inplace=True, errors='ignore')
    qa_df.drop([i for i in qa_df.columns if (('m(rt)' in i)|('sd(rt)' in i)) & (('SS:stop' in i)|('GNG:nogo' in i))], axis=1, inplace=True, errors='ignore')
    #others:
    if ('flanker_with_cued_task_switching' in exp_id) or ('shape_matching_with_cued_task_switching' in exp_id):
        qa_df.columns = qa_df.columns.str.replace(r'switch_new','switch')
    qa_df.loc['mean'] = round(qa_df.mean(),2)
    if 'no_go' in exp_id: exp_id = exp_id.replace('no_go','nogo')
    #writing: use the first two lines to write normally,
    #or the last two lines if writing QA files with 0.5 error added where accuracy==1 (for EZ-Diffusion model)
#     if not os.path.exists(explore_dir + 'QA/'): os.makedirs(explore_dir + 'QA/')
#     qa_df.to_csv(explore_dir + 'QA/' + exp_id + '_qa.csv')
    if not os.path.exists(explore_dir + 'QA_add0.5error_forACC=1/'): os.makedirs(explore_dir + 'QA_add0.5error_forACC=1/')
    qa_df.to_csv(explore_dir + 'QA_add0.5error_forACC=1/' + exp_id + '_qa.csv')

n_back_with_cued_task_switching
